$\hat{\sigma_{2}}(\mathbb{P}^{1} \times \mathbb{P}^{2} \times \mathbb{P}^{2})$

In [258]:
import numpy as np

We note that we are using the base field `QQ` for all computations currently

In [259]:
# Let us initialize constants and vector spaces.


N = 18
d = 3
indeterminates = [var("".join(["x_", str(i), str(j), str(k)])) for i in range(2) for j in range(3) for k in range(3)]
R = PolynomialRing(QQ, indeterminates)
R.inject_variables()

glN = MatrixSpace(QQ, N)
dimV = binomial(N + d - 1, d)
V = VectorSpace(QQ, dimV)

Defining x_000, x_001, x_002, x_010, x_011, x_012, x_020, x_021, x_022, x_100, x_101, x_102, x_110, x_111, x_112, x_120, x_121, x_122


In [260]:
# This cell defines the explicit isomorphism between V <=> R^d

# construct monomial list
i = 0
elem = 0
lst = []
while i < N*d:
    lst.append(indeterminates[elem])
    i += 1
    if i % d == 0:
        elem += 1
S = Subsets(lst, d, submultiset=True).list()
monomials = [R(np.prod(S[i])) for i in range(dimV)]

# R^d => V
def polynomial_to_vector(p):
    if not p in R:
        return 0
    coefficient_list = []
    for i in range(len(monomials)):
        coefficient_list.append(p.monomial_coefficient(monomials[i]))
    return V(coefficient_list)

# V => R^d
def vector_to_polynomial(v):
    if not v in V:
        return 0
    p = 0
    for i, item in enumerate(Sequence(monomials)):
        p += v[i]*item
    return R(p)

In [261]:
# Data processing cell to flatten sparse vectors over V. We also create an optimized inner product function on such vectors.

# V => V
def sparsify(v):
    flattened_list = []
    for index, value in enumerate(v):
        if v[index] != 0:
            flattened_list.append((index,value))
    return flattened_list


# <V,V> => QQ
def sparse_inner_product(a,b):
    i = 0; j = 0
    result = 0
    while i < len(a) and j < len(b):
        if a[i][0] == b[j][0]:
            result += a[i][1]*b[j][1]
            i += 1
            j += 1
        elif a[i][0] < b[j][0]:
            i += 1
        else:
            j += 1
    return result

In [262]:
# construct f

M_6by3 = MatrixSpace(R, 6, 3)

x_sub = lambda i,j,k: R(var("".join(["x_", str(i), str(j), str(k)])))

A_12 = M_6by3(0)
A_23 = M_6by3(0)

for i in range(2):
    for j in range(3):
        for k in range(3):
            A_12[3*i + j, k] = x_sub(i,j,k)
            A_23[3*i + k, j] = x_sub(i,j,k)

f = A_12.minors(3) + A_23.minors(3)

In [263]:
f_vec = [polynomial_to_vector(fi) for fi in f]

# CHECK: three determinant polynomials are linearly dependent
print(len(V.linear_dependence(f_vec)) > 0)

True


In [264]:
# define W = span{f}, W_perp = orthogonal complement of span{f}

W = V.subspace(f_vec)

f_vec_basis = W.basis() # take only the linearly independent ones
grad_f = [vector_to_polynomial(fi).gradient() for fi in f_vec_basis]

W_perp = W.complement()

print(W)
print(W_perp)

g = W_perp.basis()
p = W.dimension()
q = W_perp.dimension()

Vector space of degree 1140 and dimension 36 over Rational Field
Basis matrix:
36 x 1140 dense matrix over Rational Field
Vector space of degree 1140 and dimension 1104 over Rational Field
Basis matrix:
1104 x 1140 dense matrix over Rational Field


In [254]:
# This defines the action of glN (general linear lie algebra) on the homogeneous polynomials of degree d in R.

# X_action: (glN, R^d) => R^d
def X_action(X, i):
    sum = 0
    for a in range(N):
        for b in range(N):
            sum += -X[a][b]*indeterminates[b]*grad_f[i][a]
    return R(sum)

def E_action(a, b, f_index):
    sum = -1*indeterminates[b]*grad_f[f_index][a]
    return R(sum)

We note `M_f` $:= \tilde{M}_f$

In [255]:
# Pre-processing memoization of entries of M_f

# returns E_ij basis matrix of glN
E = lambda i,j: list(glN.basis())[(N*i) + j]

# E_action_f[l][i][j] = (E_ij)(f_l)

E_action_f = [[[0 for j in range(N)] for i in range(N)] for l in range(p)]
for l in range(p):
    print(l)
    for i in range(N):
        for j in range(N):
            E_action_f[l][i][j] = sparsify(polynomial_to_vector(E_action(i,j,l)))

# convert g to sparse representation
g_sparse = [sparsify(g[i]) for i in range(q)]

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35


In [256]:
# Construct M_f
M_f = zero_matrix(QQ, p*q, N^2)
for l in range(p):
    for k in range(q):
        for i in range(N):
            for j in range(N):
                M_f[(p*l) + k,(N*i) + j] = sparse_inner_product(E_action_f[l][i][j], g_sparse[k])

In [257]:
print('rank(M_f) =', M_f.rank())
print('dim(ker(M_f)) =', glN.dimension() - M_f.rank())

rank(M_f) = 142
dim(ker(M_f)) = 182
